# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [5]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [6]:
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [32]:
# Aggregate amount_spent for unique customers -> group by
amount_orders = orders.groupby(['CustomerID']).agg({'amount_spent': sum}).reset_index()
amount_orders

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [33]:
amount_orders.sort_values('amount_spent', ascending = False)

,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06
...,...,...
4099,17956,12.75
3015,16454,6.90
1794,14792,6.20
3218,16738,3.75


In [36]:
# select customers in a given quantile range (0.75 to 1)
q75 = np.percentile(amount_orders['amount_spent'],75)
q95 = np.percentile(amount_orders['amount_spent'],95)

print(q75)
print(q95)

1661.64
5840.181999999982


In [37]:
top75 = amount_orders[(amount_orders['amount_spent']>q75)]
top75
# in this dataframe, we have only VIP and Preferred Customers

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
5,12352,2506.04
...,...,...
4319,18259,2338.60
4320,18260,2643.20
4328,18272,3078.58
4337,18283,2094.88


In [48]:
# label VIP and Preferred Customers

def type_client(x) :
    if x >= 5840.181999999982 :
        return 'VIP'
    else : 
        return 'Preferred'

In [50]:
top75['Type'] = top75['amount_spent'].apply(type_client)
top75.head()

/var/folders/tk/75w8q4_s16n1_zv4jdmdlwhh0000gn/T/ipykernel_36331/1582641288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top75['Type'] = top75['amount_spent'].apply(type_client)


,CustomerID,amount_spent,Type
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
5,12352,2506.04,Preferred


In [52]:
top75['Type'].value_counts()

Preferred    868
VIP          217
Name: Type, dtype: int64

In [56]:
top75.shape

(1085, 3)

In [ ]:
# we could merge this top75 dataframe with the full one...

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [59]:
# first let's merge the full dataframe and the Top75 one to get back all infos from VIP and Preferred customers
cust_details = pd.merge(left = top75, 
                                 right = orders, 
                                 how = 'inner', 
                                 left_on = "CustomerID", 
                                 right_on= "CustomerID") 
display(cust_details.shape)
cust_details.head()

(256265, 16)

,CustomerID,amount_spent_x,Type,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,Country,amount_spent_y
0,12346,77183.6,VIP,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,United Kingdom,77183.6
1,12347,4310.0,Preferred,14938,537626,85116,2010,12,2,14,black candelabra t-light holder,12,2010-12-07 14:57:00,2.10,Iceland,25.2
2,12347,4310.0,Preferred,14939,537626,22375,2010,12,2,14,airline bag vintage jet set brown,4,2010-12-07 14:57:00,4.25,Iceland,17.0
3,12347,4310.0,Preferred,14940,537626,71477,2010,12,2,14,colour glass. star t-light holder,12,2010-12-07 14:57:00,3.25,Iceland,39.0
4,12347,4310.0,Preferred,14941,537626,22492,2010,12,2,14,mini paint set vintage,36,2010-12-07 14:57:00,0.65,Iceland,23.4


In [74]:
# Let's group by customers again (many IDs are repeated)
cust_country = cust_details.groupby(['CustomerID','Type','Country']).agg({'amount_spent_y': sum}).reset_index()
cust_country

,CustomerID,Type,Country,amount_spent_y
0,12346,VIP,United Kingdom,77183.60
1,12347,Preferred,Iceland,4310.00
2,12348,Preferred,Finland,1797.24
3,12349,Preferred,Italy,1757.55
4,12352,Preferred,Norway,2506.04
...,...,...,...,...
1086,18259,Preferred,United Kingdom,2338.60
1087,18260,Preferred,United Kingdom,2643.20
1088,18272,Preferred,United Kingdom,3078.58
1089,18283,Preferred,United Kingdom,2094.88


In [81]:
# now let's slide the dataframe with only VIP customers
VIP = cust_country[cust_country['Type'] == 'VIP']
VIP

,CustomerID,Type,Country,amount_spent_y
0,12346,VIP,United Kingdom,77183.60
6,12357,VIP,Switzerland,6207.67
7,12359,VIP,Cyprus,6372.58
24,12409,VIP,Switzerland,11072.67
25,12415,VIP,Australia,124914.53
...,...,...,...,...
1059,18109,VIP,United Kingdom,8052.97
1062,18139,VIP,United Kingdom,8438.34
1065,18172,VIP,United Kingdom,7561.68
1074,18223,VIP,United Kingdom,6484.54


In [83]:
# now let's find the number of VIP for each country 
VIP.pivot_table(index= ['Country'], values = ['Type'], aggfunc = pd.Series.value_counts).sort_values('Type', ascending = False)

,Type
Country,
United Kingdom,177
Germany,10
France,9
Switzerland,3
Spain,2
Portugal,2
Australia,2
EIRE,2
Japan,2


In [84]:
# The United Kingdom is the country with the highest number of VIP customers (then Germany, then France)

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# to answer this question, we can go back on dataframe cust_country

In [85]:
cust_country.pivot_table(index= ['Country'], values = ['CustomerID'], aggfunc = pd.Series.nunique).sort_values('CustomerID', ascending = False)

,CustomerID
Country,
United Kingdom,932
Germany,39
France,29
Belgium,12
Switzerland,9
Spain,9
Portugal,7
Norway,7
Italy,5


In [ ]:
# The UK are by far the country with the highest numbers of clients (VIP+Preferred)